In [10]:
import requests
import pandas as pd
import os
from dotenv import load_dotenv
import sqlite3

In [11]:
db_path = os.path.join(os.path.dirname(os.getcwd()), 'data/financials.db')
con = sqlite3.connect(db_path)
cur = con.cursor()

In [ ]:
cur.execute("""
CREATE TABLE IF NOT EXISTS economic_indicators (
    id INTEGER PRIMARY KEY,
    pub_date TEXT,
    full_text TEXT,
    combined_text TEXT,
    source REAL,
    web_url TEXT
)
""")
con.commit()

In [3]:
load_dotenv()
key = os.getenv("ALPHA_VANTAGE_KEY")

### Functions 
- TIME_SERIES_MONTHLY
- NEWS_SENTIMENT
- INCOME_STATEMENT
- BALANCE_SHEET
- CASH_FLOW
- EARNINGS
- EARNINGS_CALENDAR
- REAL_GDP
- REAL_GDP_PER_CAPITA
- TREASURY_YIELD
- FEDERAL_FUNDS_RATE
- UNEMPLOYMENT

In [8]:
ECONOMIC_FUNCTIONS = [
    'FEDERAL_FUNDS_RATE',
    'TREASURY_YIELD',
    'REAL_GDP_PER_CAPITA',
    'REAL_GDP',
    'CPI',
    'INFLATION',
    'RETAIL_SALES',
    'DURABLES',
    'UNEMPLOYMENT',
    'NONFARM_PAYROLL'
]

In [14]:
ALPHA_VANTAGE_URL = "https://www.alphavantage.co/query"


In [60]:
def get_economic_data(func, api_key, **kwargs):
    for key in kwargs.keys():
        if key not in ('interval','maturity'):
            raise ValueError(f"kwarg {key} not accepted")
    url = "https://www.alphavantage.co/query"
    params = {
        'function': func,
        'apikey': api_key
    }
    params.update(kwargs)
    
    if 'maturity' in params:
        if func != 'TREASURY_YIELD':
            params.pop('maturity')
        else:
            maturities = ('3month', '2year', '5year', '7year', '10year', '30year')
            if params['maturity'] not in maturities:
                raise ValueError(f"Maturity has to be one of {maturities}")
    if 'interval' in params:
        if func in ('FEDERAL_FUNDS_RATE', 'TREASURY_YIELD'):
            intervals = ('daily', 'weekly', 'monthly')
        elif func == 'REAL_GDP':
            intervals = ('quarterly', 'annual')
        elif func == 'CPI':
            intervals = ('monthly', 'semiannual')
        else:
            raise RuntimeError(f"interval provided for a Economic Function that doesn't need it: {func}")
        if params['interval'] not in intervals:
            raise ValueError(f"Interval for {func} has to be one of {intervals}")

    resp = requests.get(url, params=params)
    if resp.status_code == 200:
        df = pd.json_normalize(
            data=resp.json(),
            record_path='data',
            meta= ['name','interval','unit']
            ).rename({'name': 'economic_indicator'},axis=1)
        return df
    else:
        raise requests.HTTPError(f"HTTP Error: {resp.status_code}")
    
    


In [59]:
get_economic_data("TREASURY_YIELD","key",maturity = "10year",interval = "daily")

{'function': 'TREASURY_YIELD',
 'apikey': 'key',
 'maturity': '10year',
 'interval': 'daily'}

In [45]:
ECONOMIC_FUNCTIONS

['FEDERAL_FUNDS_RATE',
 'TREASURY_YIELD',
 'REAL_GDP_PER_CAPITA',
 'REAL_GDP',
 'CPI',
 'INFLATION',
 'RETAIL_SALES',
 'DURABLES',
 'UNEMPLOYMENT',
 'NONFARM_PAYROLL']

In [16]:
test = get_economic_data(ECONOMIC_FUNCTIONS[0], api_key=key)

In [22]:
data = test.json()

In [61]:
df = pd.json_normalize(
    data=data,
    record_path='data',
    meta= ['name','interval','unit'],
    
).rename({'name': 'economic_indicator'},axis=1)

In [9]:
data = {}

for func in ECONOMIC_FUNCTIONS:
    print(func)

FEDERAL_FUNDS_RATE
TREASURY_YIELD
REAL_GDP_PER_CAPITA
REAL_GDP
CPI
INFLATION
RETAIL_SALES
DURABLES
UNEMPLOYMENT
NONFARM_PAYROLL
